In [1]:
import sys
print(sys.version)
print(spark.version)

3.8.5 | packaged by conda-forge | (default, Aug 29 2020, 01:22:49) 
[GCC 7.5.0]
3.0.1


In [2]:
import subprocess

from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [4]:
%%time

influencers = spark.read.parquet('gs://msca-bdp-students-bucket/shared_data/abharathsingh/influencers')

CPU times: user 1.04 ms, sys: 3.29 ms, total: 4.32 ms
Wall time: 5.86 s


In [5]:
influencers.approxQuantile("avg_influence", [0.5], 0) #media influence score for finding top 50% of influencers

[0.2351915994450337]

In [4]:
%%time

covid_influence = spark.read.parquet('gs://msca-bdp-students-bucket/shared_data/abharathsingh/covid_influence')

CPU times: user 4.74 ms, sys: 0 ns, total: 4.74 ms
Wall time: 6.89 s


In [6]:
#covid_influence.count()

In [7]:
from pyspark.sql import functions as f

In [8]:
covid_influence2 = covid_influence.withColumn("Organization", 
                        f.when(covid_influence.verified_user == 'false', 'Other') \
                        .when((covid_influence.user_description.contains('health')) | (covid_influence.user_name.contains('health')) \
                              | (covid_influence.user_description.contains('medic')) | (covid_influence.user_name.contains('medic')) \
                              | (covid_influence.user_description.contains('clinic')) | (covid_influence.user_name.contains('clinic')) \
                              | (covid_influence.user_description.contains('nurse')) | (covid_influence.user_name.contains('nurse')) \
                              | (covid_influence.user_description.contains('doctor')) | (covid_influence.user_name.contains('doctor')) \
                              | (covid_influence.user_description.contains('hospital')) | (covid_influence.user_name.contains('hospital')) \
                              | (covid_influence.user_description.contains('disease')) | (covid_influence.user_name.contains('disease')), 'Health') \
                        .when((covid_influence.user_description.contains('gov')) | (covid_influence.user_name.contains('gov')) \
                              | (covid_influence.user_description.contains('president')) | (covid_influence.user_name.contains('president')) \
                              | (covid_influence.user_description.contains('senat')) | (covid_influence.user_name.contains('senat')) \
                              | (covid_influence.user_description.contains('congress')) | (covid_influence.user_name.contains('congress')) \
                              | (covid_influence.user_description.contains('prime minister')) | (covid_influence.user_name.contains('prime minister')) \
                              | (covid_influence.user_description.contains('parliament')) | (covid_influence.user_name.contains('parliament')) \
                              | (covid_influence.user_description.contains('vp')) | (covid_influence.user_name.contains('vp')) \
                              | (covid_influence.user_description.contains('white house')) | (covid_influence.user_name.contains('white house')) \
                              | (covid_influence.user_description.contains('mayor')) | (covid_influence.user_name.contains('mayor')) \
                              | (covid_influence.user_description.contains('democr')) | (covid_influence.user_name.contains('democr')) \
                              | (covid_influence.user_description.contains('republic')) | (covid_influence.user_name.contains('republic')), 'Government') \
                        .when((covid_influence.user_description.contains('news')) | (covid_influence.user_name.contains('news')) \
                              | (covid_influence.user_description.contains('daily')) | (covid_influence.user_name.contains('daily')) \
                              | (covid_influence.user_description.contains('times')) | (covid_influence.user_name.contains('times')) \
                              | (covid_influence.user_description.contains('journal')) | (covid_influence.user_name.contains('journal')), 'News') \
                        .when(((covid_influence.user_description.contains('actor')) & (covid_influence.followers_count > 30000)) \
                              | ((covid_influence.user_description.contains('singer')) & (covid_influence.followers_count > 30000)) \
                              | ((covid_influence.user_description.contains('writer')) & (covid_influence.followers_count > 30000)) \
                              | ((covid_influence.user_description.contains('play')) & (covid_influence.followers_count > 30000)) \
                              | ((covid_influence.user_description.contains('musician')) & (covid_influence.followers_count > 30000)) \
                              | ((covid_influence.user_description.contains('actress')) & (covid_influence.followers_count > 30000)) \
                              | ((covid_influence.user_description.contains('golfer')) & (covid_influence.followers_count > 30000)) \
                              | ((covid_influence.user_description.contains('author')) & (covid_influence.followers_count > 30000)), 'Celebrity') \
                        .when(covid_influence.followers_count >= 10000, 'Influencer')) 

In [9]:
covid_influence2 = covid_influence2.na.fill(value='Other',subset=["Organization"])

In [11]:
#covid_influence2

In [16]:
covid_influence2.count()

13672943

## data for top 50% of influencers based on their average influence score

In [17]:
top_50 = covid_influence2.filter(covid_influence2.avg_influence >= 0.2351915994450337)

In [19]:
#top_50

In [20]:
top_50.count()

11009056

In [25]:
top_50.write.format("parquet")\
.mode('overwrite')\
.save('gs://msca-bdp-students-bucket/shared_data/abharathsingh/top_50')

print('done')

done


## get organization counts for all covid twitterers

In [12]:
covid_influence3 = covid_influence2.dropDuplicates(['user_id'])

In [14]:
#covid_influence3

In [15]:
covid_influence3.count()

2729351

In [16]:
covid_influence3.select(['Organization']).groupby("Organization").count()

Organization,count
Government,6081
Influencer,16841
Health,3862
Other,2684856
Celebrity,1583
News,16128
